<div class="frontmatter text-center">
<h1>Geospatial Data Science</h1>
<h2>Lecture 9: Spatial networks</h2>
<h3>IT University of Copenhagen, Spring 2022</h3>
<h3>Instructor: Michael Szell</h3>
</div>

# Source
This notebook was adapted from:

* OSMnx examples: https://github.com/gboeing/osmnx-examples/blob/main/notebooks/06-stats-indicators-centrality.ipynb
* Spaghetti: https://pysal.org/spaghetti/notebooks/network-spatial-dependence.html

In [ ]:
import geopandas as gpd
import osmnx as ox
import numpy as np
import networkx as nx
import pandas as pd

from haversine import haversine, haversine_vector

import matplotlib.pyplot as plt
from scipy.spatial import Delaunay, delaunay_plot_2d, Voronoi, voronoi_plot_2d

import matplotlib
import spaghetti
import matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar

%matplotlib inline
ox.__version__

# Haversine distance

Always use the Haversine distance when calculating distances between far away points: https://github.com/mapado/haversine

In [ ]:
copenhagen = (55.67594, 12.56553)  # (lat, lon)
london = (51.509865, -0.118092)
sanfrancisco = (37.773972, -122.431297)
newyork = (40.730610, -73.935242)

In [ ]:
haversine(copenhagen, sanfrancisco)

In [ ]:
haversine_vector(3*[copenhagen], [london, newyork, sanfrancisco])

# Use OSMnx to calculate street network indicators

First we download the Frederiksberg drive network with OSMnx:

In [ ]:
place = "Frederiksberg Municipality"
G = ox.graph_from_place(place, network_type="drive")
G_proj = ox.project_graph(G)

## Calculate basic street network measures (topological and geometric)

In [ ]:
# Calculate Frederiksberg's basic stats, then show the average circuity
stats = ox.basic_stats(G)
stats["circuity_avg"]

To calculate density-based stats, you must also pass the network's bounding area in square meters (otherwise basic_stats() will just skip them in the calculation):

In [ ]:
# get the street network for a place, and its area in square meters
city = ox.geocode_to_gdf(place)
city_proj = ox.project_gdf(city)
area = city_proj.unary_union.area
area

In [ ]:
# calculate basic and extended network stats, merge them together, and display
stats = ox.basic_stats(G, area=area)
pd.Series(stats)

Streets/intersection counts and proportions are nested dicts inside the stats dict. To convert these stats to a pandas dataframe (to compare/analyze multiple networks against each other), just unpack these nested dicts first:

In [ ]:
# unpack dicts into individiual keys:values
stats = ox.basic_stats(G, area=area)
for k, count in stats["streets_per_node_counts"].items():
    stats["{}way_int_count".format(k)] = count
for k, proportion in stats["streets_per_node_proportions"].items():
    stats["{}way_int_prop".format(k)] = proportion

# delete the no longer needed dict elements
del stats["streets_per_node_counts"]
del stats["streets_per_node_proportions"]

# load as a pandas dataframe
pd.DataFrame(pd.Series(stats, name="value")).round(3)

### Inspect betweenness centrality

In [ ]:
# calculate betweenness with a digraph of G (ie, no parallel edges)
bc = nx.betweenness_centrality(ox.get_digraph(G), weight="length")
max_node, max_bc = max(bc.items(), key=lambda x: x[1])
max_node, max_bc

In Frederiksberg Municipality, the node with the highest betweenness centrality has 25% of all shortest paths running through it. Let's highlight it in the plot:

In [ ]:
nc = ["r" if node == max_node else "w" for node in G.nodes]
ns = [150 if node == max_node else 20 for node in G.nodes]
fig, ax = ox.plot_graph(G, node_size=ns, node_color=nc, node_zorder=2, bgcolor="k")

~25% of all shortest paths run through the node highlighted in red. Let's look at the relative betweenness centrality of every node in the graph:

In [ ]:
# add the betweenness centrality values as new node attributes, then plot
nx.set_node_attributes(G, bc, "bc")
nc = ox.plot.get_node_colors_by_attr(G, "bc", cmap="viridis")
fig, ax = ox.plot_graph(
    G,
    node_color=nc,
    node_size=40,
    node_zorder=2,
    edge_linewidth=0.2,
    edge_color="w",
    bgcolor="k",
)

Above, the nodes are visualized by betweenness centrality, from low (dark violet) to high (light yellow). The colors in the colorspace are linearly mapped to the attribute values.

# Delaunay triangulation and Voronoi diagram

Let's showcase the Delaunay traingulation and the Voronoi diagram for pharmacies in Frederiksberg. In particular, the Voronoi diagram will tell us: What is the closest pharmacy to any location in Frederiksberg?

In [ ]:
# Let's fetch all pharmacies and project them
tags = {'amenity': ['pharmacy']}
gdf_pharmacies = ox.geometries_from_place(place, tags=tags)
gdf_pharmacies = ox.project_gdf(gdf_pharmacies)
gdf_pharmacies

In [ ]:
# Let's extract the coordinates
pharmacies_coords = np.vstack((
    np.array(gdf_pharmacies.geometry.x),
    np.array(gdf_pharmacies.geometry.y)
)).T
pharmacies_coords

## Delaunay triangulation

In [ ]:
tri = Delaunay(pharmacies_coords)
fig = plt.figure(figsize=(12, 9))
axes = fig.add_axes([0, 0, 1, 1])
delaunay_plot_2d(tri, ax=axes);

# Plot city border
city_proj.plot(fc="#F6F6F6", ec="none", ax=axes);

# Plot street network
ox.plot_graph(G_proj, node_size=0, bgcolor="w", ax=axes, edge_color="#CCCCCC");

## Voronoi diagram

In [ ]:
vor = Voronoi(pharmacies_coords)
fig = plt.figure(figsize=(12, 9))
axes = fig.add_axes([0, 0, 1, 1])
voronoi_plot_2d(vor, ax=axes, line_width = 3, point_size=20);

# Plot city border
city_proj.plot(fc="#F6F6F6", ec="none", ax=axes);

# Plot street network
ox.plot_graph(G_proj, node_size=0, bgcolor="w", ax=axes, edge_color="#CCCCCC");

Voila! The thick black lines show us the areas closest to each pharmacy (blue).

# Network-constrained spatial clustering with Ripley's K using Spaghetti

Ripley's K function takes a point pattern and considers all pairwise distances of nearest neighbors to determine the existence of clustering, or lack thereof, over a delineated range of distances: https://en.wikipedia.org/wiki/Spatial_descriptive_statistics#Ripley's_K_and_L_functions

However, using Ripley's K for urban amenities is wrong: Amenities cannot be reached as the crow flies -you can't move through buildings-, but only over the street network. The Spaghetti package is part of PySAL - it allows to constrain spatial dependence to a street network: https://pysal.org/spaghetti/notebooks/network-spatial-dependence.html It combines point pattern analysis with spatial networks. It has tools to snap points to a network and to analyze point patterns constrained to the network, which can be very useful for asking questions like "How clustered are pubs/pharmacies *on the street network*?"

We want to analyze two point of interest (POI) datasets in Frederiksberg: 1) Pubs+Restaurants, and 2) Pharmacies (from above). Let's fetch and project the new dataset, pubs+restaurants:

In [ ]:
tags = {'amenity': ['pub', 'restaurant']}
gdf_pubs = ox.geometries_from_place(place, tags=tags)
gdf_pubs = ox.project_gdf(gdf_pubs)
gdf_pubs

In [ ]:
fig = plt.figure(figsize=(12, 9))
axes = fig.add_axes([0, 0, 1, 1])

# Plot city border
city_proj.plot(fc="#F6F6F6", ec="none", ax=axes);

# Plot the two POI datasets
gdf_pubs.plot(ax=axes, color="r", markersize=60, marker="s")
gdf_pharmacies.plot(ax=axes, color="b", markersize=120)

# Plot street network
ox.plot_graph(G_proj, node_size=0, bgcolor="w", ax=axes);

We first need a helper function for nice plotting (adapted from https://pysal.org/spaghetti/notebooks/network-spatial-dependence.html#Results-plotting-helper-function)

In [ ]:
def plot_k(k, _arcs, df1, df2, obs, scale=True, wr=[1, 1.2], size=(14, 7)):
    """Plot a Global Auto K-function and spatial context."""
    def function_plot(f, ax):
        """Plot a Global Auto K-function."""
        ax.plot(k.xaxis, k.observed, "b-", linewidth=1.5, label="Observed")
        ax.plot(k.xaxis, k.upperenvelope, "r--", label="Upper")
        ax.plot(k.xaxis, k.lowerenvelope, "k--", label="Lower")
        ax.legend(loc="best", fontsize="x-large")
        title_text = "Global Auto $K$ Function: %s\n" % obs
        title_text += "%s steps, %s permutations," % (k.nsteps, k.permutations)
        title_text += " %s distribution" % k.distribution
        f.suptitle(title_text, fontsize=25, y=1.1)
        ax.set_xlabel("Distance $(r)$", fontsize="x-large")
        ax.set_ylabel("$K(r)$", fontsize="x-large")

    def spatial_plot(ax):
        """Plot spatial context."""
        base = _arcs.plot(ax=ax, color="k", alpha=0.25)
        df1.plot(ax=base, color="g", markersize=30, alpha=0.25)
        df2.plot(ax=base, color="g", marker="x", markersize=100, alpha=0.5)
        carto_elements(base, scale)

    sub_args = {"gridspec_kw":{"width_ratios": wr}, "figsize":size}
    fig, arr = matplotlib.pyplot.subplots(1, 2, **sub_args)
    function_plot(fig, arr[0])
    spatial_plot(arr[1])
    fig.tight_layout()

def carto_elements(b, scale):
    """Add/adjust cartographic elements."""
    if scale:
        kw = {"units":"m", "dimension":"si-length", "fixed_value":1000}
        b.add_artist(ScaleBar(1, **kw))
    b.set(xticklabels=[], xticks=[], yticklabels=[], yticks=[]);

Because spaghetti has no OSMnx integration, we need to use shapefiles for example. We therefore export the graph, so that we can import it later as a spaghetti.Network

In [ ]:
ox.io.save_graph_shapefile(G_proj, "Frederiksberg")

Now we build the spaghetti.Network from the shape file:

In [ ]:
ntw = spaghetti.Network(in_data="Frederiksberg/edges.shp")
vertices_df, arcs_df = spaghetti.element_as_gdf(
    ntw, vertices=ntw.vertex_coords, arcs=ntw.arcs
)

Snapping pubs and pharmacies to our street network, creating two pointpatterns on the network:

In [ ]:
ntw.snapobservations(gdf_pubs, "pubs", attribute=True)
ntw.snapobservations(gdf_pharmacies, "pharmacies", attribute=True)
ntw.pointpatterns

Saving the pointpatterns and their snapped versions as gdf, for plotting later:

In [ ]:
pubs = spaghetti.element_as_gdf(ntw, pp_name="pubs")
pubs_snapped = spaghetti.element_as_gdf(ntw, pp_name="pubs", snapped=True)
pharmacies = spaghetti.element_as_gdf(ntw, pp_name="pharmacies")
pharmacies_snapped = spaghetti.element_as_gdf(ntw, pp_name="pharmacies", snapped=True)

## Clustering of pubs and restaurants

Running Ripley's K for pubs *on the network* (this can take around 2 minutes):

In [ ]:
np.random.seed(0)
kres = ntw.GlobalAutoK(
    ntw.pointpatterns["pubs"],
    nsteps=10,
    permutations=99) # Keep permutations low here, otherwise too much time needed for computation
plot_k(kres, arcs_df, pubs, pubs_snapped, "pubs")

### Interpretation

Because the observed curve is always above the simulation envelope, pubs+restaurants are clustered on the street network **on all scales**! 

It looks like a showcase for economies of agglomeration or business cluster effects, especially Gammel Kongevej and Pile Alle/Falkoner Alle:  
https://en.wikipedia.org/wiki/Economies_of_agglomeration https://en.wikipedia.org/wiki/Business_cluster#Cluster_effect

## Clustering of pharmacies

Now let's run Ripley's K on the pharmacies. This is much faster due to the low number of points, so we can crank up the parameters:

In [ ]:
np.random.seed(0)
kres = ntw.GlobalAutoK(
    ntw.pointpatterns["pharmacies"],
    nsteps=100,
    permutations=999)
plot_k(kres, arcs_df, pharmacies, pharmacies_snapped, "pharmacies")

### Interpretation

Pharmacies show different clustering behavior than pubs+restaurants. If it wasn't for those two pharmacies in the north east right next to each other, their observed curve would be on the lowest end of the simulation envelope for low distances, showing that they are quite dispersed on short ranges. Maybe there are minimum distance laws in place to make pharmacies cover the city well? On the other hand, for distances above 2km, the observed curve is above the envelope, meaning clustering. On this scale, pharmacies are clustered in the east, and there is quite some empty space in the north west and south west of Frederiksberg.